In [1]:
import numpy as np
import os

from scipy.spatial import distance
import matplotlib.pyplot as plt
import networkx as nx
import copy
from scipy import stats
from copy import deepcopy
import time
import re

In [2]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)


In [3]:
def extract_position(fname):
    neuron_data = np.loadtxt(fname)
    #print(fname)
    #print(neuron_data.shape)
    if neuron_data.ndim == 1:#  there exists files which do not have any neurites
        return np.zeros((1,3)), neuron_data[2:5]
    #else:
    soma = neuron_data[0,2:5] # considering only the positions/coordinates
    #print(soma)
    neurites = neuron_data[1:,2:5]
    return neurites,soma # neurites refers to either axon terminals/dendrites. It is a loose term.


In [4]:
def count_connections(axonTerminalsA, dendritesB, synaptic_cleft):
    # cdist is used to calculate euclidean distance between axon terminals and neurites
    # dist_axonTerminalsA_dendritesB is 2d array of dimension nrows(axonTerminalsA) x nrows(dendritesB)
    dist_axonTerminalsA_dendritesB = distance.cdist(axonTerminalsA, dendritesB)
    #count all the dist less than 3um as a connection btw axon terminal A and dendrite B
    connections = np.count_nonzero(dist_axonTerminalsA_dendritesB <= synaptic_cleft)
    return connections


In [6]:
def extract_adjacency(file_path_axon, file_path_dendrite, synaptic_cleft):

    file_list_axon=[f for f in sorted_alphanumeric(os.listdir(file_path_axon)) if f.endswith('.swc')
                and os.path.isfile(os.path.join(file_path_axon, f))]
    file_list_dendrite=[f for f in sorted_alphanumeric(os.listdir(file_path_dendrite)) if f.endswith('.swc')
                        and os.path.isfile(os.path.join(file_path_dendrite, f))]# redundant but simply taken

    neuron_names = []
    n_neurons = len(file_list_axon)
    adjacency = np.zeros((n_neurons,n_neurons))
    
    start = time.time()
    counter = 0 
    
    
    for i in range(n_neurons):
        j_list = list(range(i+1, n_neurons))

        for j in j_list:

            axonTerminalsA, somaA = extract_position(file_path_axon+file_list_axon[i])
            dendritesB, somaB = extract_position(file_path_dendrite+file_list_dendrite[j])

            axonTerminalsB, somaB = extract_position(file_path_axon+file_list_axon[j])
            dendritesA, somaA = extract_position(file_path_dendrite+file_list_dendrite[i])


            if np.all(axonTerminalsA==0):
                adjacency[i,j] = 0
            else:
                adjacency[i,j] = count_connections(axonTerminalsA, dendritesB, synaptic_cleft)

            if np.all(axonTerminalsB==0):
                adjacency[j,i] = 0
            else:
                adjacency[j,i] = count_connections(axonTerminalsB, dendritesA, synaptic_cleft)
        
        counter+=1
        
        if (counter%100)==0:
            end=time.time()
            print(i, "%.3f min"%((end-start)/60))
            start = time.time()
                
        neuron_names.append(file_list_axon[i][:-4])

    return adjacency, neuron_names


In [7]:
file_path_axon = "./dendrites_axonTermials/axon_terminals/"
file_path_dendrite = "./dendrites_axonTermials/input_dendrites/"
synaptic_cleft=5

adjacency, neuron_names = extract_adjacency(file_path_axon, file_path_dendrite, synaptic_cleft=synaptic_cleft)


99 19.634 min
199 26.073 min
299 12.588 min
399 9.431 min
499 9.197 min
599 10.042 min
699 10.306 min
799 10.157 min
899 9.503 min
999 13.650 min
1099 10.901 min
1199 15.235 min
1299 11.785 min
1399 9.739 min
1499 8.823 min
1599 8.878 min
1699 9.227 min
1799 7.416 min
1899 5.989 min
1999 4.864 min
2099 5.033 min
2199 4.316 min
2299 3.744 min
2399 3.778 min
2499 3.276 min
2599 3.090 min
2699 2.579 min
2799 2.129 min
2899 1.718 min
2999 1.190 min
3099 0.605 min


In [8]:
np.save("adjacency_synaptic_cleft_5.000um.npy", adjacency.astype(np.int64))
